In [1]:
from pathlib import Path

folder = Path('.')
target = folder / 'collage.pdb'
assert target.exists()

In [2]:
import pyrosetta
import pyrosetta_help as ph
from types import ModuleType
from IPython.display import display, HTML

prc: ModuleType = pyrosetta.rosetta.core
prp: ModuleType = pyrosetta.rosetta.protocols
prcc: ModuleType = pyrosetta.rosetta.core.conformation
pr_scoring: ModuleType = pyrosetta.rosetta.core.scoring

logger = ph.configure_logger()
pyrosetta.distributed.maybe_init(extra_options=ph.make_option_string(no_optH=False,
                                                ex1=None,
                                                ex2=None,
                                                #mute='all',
                                                ignore_unrecognized_res=True,
                                                load_PDB_components=False,
                                                ignore_waters=True)
                               )

/data/xchem-fragalysis/mferla/waconda/lib/python3.11/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [4]:
pyrosetta.rosetta.basic.options.set_boolean_option('run:ignore_zero_occupancy', False)
pyrosetta.rosetta.basic.options.set_boolean_option('in:auto_setup_metals', True)
pose: pyrosetta.Pose = pyrosetta.pose_from_file(str(target.absolute()))

In [5]:
from collections import Counter

con_tally: dict = Counter([type(con).__name__ for con in pose.constraint_set().get_all_constraints()]).most_common()

print('chains', prc.pose.conf2pdb_chain(pose))
print('sequence', pose.sequence())
print('Constraints present', con_tally)

chains map_unsigned_long_char{1: A, 2: B}
sequence DMYIERAGDITWEKDAEVTGNSPRLDVALDESGDFSLVEGETTDGVYRVMTRRLLGSTQVGVGVMQEGVFHTMWHVTKGAALRSGEGRLDPYWGDVKQDLVSYCGPWKLDAAWDGLSEVQLLAVPPGERAKNIQTLPGIFKTKDGDIGAVALDYPAGTSGSPILDKCGRVIGLYGNGVVIKN
Constraints present []


In [6]:
scorefxn: pr_scoring.ScoreFunction = pyrosetta.get_fa_scorefxn()
scorefxn.set_weight(pr_scoring.ScoreType.coordinate_constraint, 20)
scorefxn.set_weight(pr_scoring.ScoreType.angle_constraint, 5)
scorefxn.set_weight(pr_scoring.ScoreType.atom_pair_constraint, 5)
pyrosetta.rosetta.basic.options.set_boolean_option('relax:constrain_relax_to_start_coords', True)
pyrosetta.rosetta.protocols.relax.FastRelax.register_options()
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, 15)
relax.constrain_relax_to_start_coords(True)
relax.apply(pose)
pose.dump_pdb('collage.strong_con.pdb')
print('x20 coordinate_constraint done')
scorefxn.set_weight(pr_scoring.ScoreType.coordinate_constraint, 5)
pyrosetta.rosetta.protocols.relax.FastRelax.register_options()
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, 15)
relax.constrain_relax_to_start_coords(True)
relax.apply(pose)
print('x5 coordinate_constraint done')
pose.dump_pdb('collage.weak_con.pdb')

x20 coordinate_constraint done
x5 coordinate_constraint done


True